# Task1


In [12]:
#import necessary libraries
import h5py
import numpy as np
import torch
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split
import torch.nn.functional as F
import torchvision
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from PIL import Image
# import torch.multiprocessing as mp
# mp.set_start_method('spawn')

In [13]:
device=torch.device("cuda:0")

In [19]:
#custom dataset class to output input and corresponding labels
class dataset(Dataset):
  def __init__(self,file):
    self.file=file
    # h5_file=h5py.File(path,'r')
    self.input=self.file["X"]
    self.label=self.file["y"]
  def __len__(self):
    self.filelength=self.input.shape[0]
    return self.filelength 
  
  def __getitem__(self,idx):
    # input1=torch.reshape(torch.from_numpy(self.input[idx][:,:,0]),(1,32,32))
    # input2=torch.reshape(torch.from_numpy(self.input[idx][:,:,1]),(32,32))
    input_stack=torch.stack([torch.from_numpy(self.input[idx][:,:,0]),torch.from_numpy(self.input[idx][:,:,1])],dim=0)
    return input_stack,self.label[idx]

In [20]:
#dataset split
electron_file=h5py.File("/home/ubuntu/Data_Task1/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5",'r')
photon_file=h5py.File("/home/ubuntu/Data_Task1/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5",'r')
data_electron=dataset(electron_file)
data_photon=dataset(photon_file)
data=data_photon+data_electron
train_data,test_data=random_split(data,[len(data)-int(0.18*len(data)),int(0.18*len(data))],generator=torch.Generator().manual_seed(42))
train_data,val_data=random_split(train_data,[len(train_data)-int(0.21875*len(train_data)),int(0.21875*len(train_data))],generator=torch.Generator().manual_seed(42))

In [21]:
#data loader
train_loader=DataLoader(train_data,batch_size=2048,shuffle=True,num_workers=4,pin_memory=True)
val_loader=DataLoader(val_data,batch_size=2048,shuffle=True,num_workers=4,pin_memory=True)
test_loader=DataLoader(test_data,batch_size=2048,shuffle=True)

In [22]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.conv1_1 = nn.Conv2d(in_channels=2, out_channels=64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.conv4_1 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.conv5_1 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(512,85)
        self.fc2 = nn.Linear(85, 85)
        self.fc3 = nn.Linear(85, 2)

    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = self.maxpool(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = self.maxpool(x)
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5) #dropout was included to combat overfitting
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.5)
        x = self.fc3(x)
        return x
net=VGG16()
net=net.train()
net=net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.00001)
print(net)

VGG16(
  (conv1_1): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_3): Conv2d(512, 51

In [18]:
#train and validation
writer = SummaryWriter()
for epoch in range(50):
    epoch_loss = 0
    epoch_accuracy = 0
    
    for data, label in train_loader:
        
        
        input = data.to(device)
        label=label.type(torch.LongTensor)
        label=label.to(device)
        
        output = net(input)
        loss = criterion(output, label)
       
        
        
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        
        acc = ((output.argmax(dim=1) == label).float().mean())
        epoch_accuracy += acc/len(train_loader)
        epoch_loss += loss/len(train_loader)
    print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch+1, epoch_accuracy,epoch_loss))
    with torch.no_grad():
        epoch_val_accuracy=0
        epoch_val_loss =0
        for data, label in val_loader:
            data = data.to(device)
            label=label.type(torch.LongTensor)
            label=label.to(device)
            val_output = net(data)
            val_loss = criterion(val_output,label)
            
            
            acc = ((val_output.argmax(dim=1) == label).float().mean())
            epoch_val_accuracy += acc/ len(val_loader)
            epoch_val_loss += val_loss/ len(val_loader)
        print('Epoch : {}, val_accuracy : {}, val_loss : {}'.format(epoch+1, epoch_val_accuracy,epoch_val_loss))
    writer.add_scalar("Loss/train", epoch_loss, epoch)
    writer.add_scalar("Loss/val", epoch_val_loss, epoch)
    writer.add_scalar("acc/train", epoch_accuracy, epoch)
    writer.add_scalar("acc/val", epoch_val_accuracy, epoch)

torch.save(net.state_dict(),"task_1.pth")

TypeError: h5py objects cannot be pickled